In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import time
import random

In [2]:
# 获取歌曲名称
song_names = pd.read_csv("all_songs.csv")['songs'].to_list()

In [3]:
# 检查用
song_names

['The Show',
 '愿与愁',
 '逆光白',
 '孤独娱乐',
 '梦不凌乱',
 '自画像',
 '谢幕',
 '如果我还剩一件事情可以做',
 '黑色泡沫',
 '你都在',
 '一时的选择',
 'Castle in the air',
 '7千3百多天',
 '愿与愁',
 'In The Joy',
 '谢幕',
 'Intro',
 'Too Bad',
 '身后',
 '想见你想见你想见你',
 '恨幸福来过',
 '慢慢喜欢你',
 'You Are The Reason',
 '咖啡+交换余生',
 '圣所',
 '生生',
 '我还年轻 我还年轻',
 '你啊你啊',
 '那女孩对我说',
 '落在生命里的光',
 '是你',
 '达尔文',
 '7千3百多天',
 '无拘',
 'From The Ashes',
 'At Least I Had You',
 "You're My Superhero",
 'Positive Procrastination',
 'Help!',
 'Better With You',
 '你還不知道?',
 'A Day At A Time',
 '一定会',
 'After The Rain',
 'At Least I Had You',
 '裹着心的光',
 'While I Can',
 'Like You Do',
 'Not Tonight',
 'All In Your Mind',
 'Head On',
 'Steady My Love',
 '过',
 '最向往的地方',
 '交换余生',
 '幸存者',
 '离开的那一些',
 '最好是',
 '暂时的记号',
 '交换余生',
 '背对背拥抱',
 'As I Believe',
 'Wonderland',
 '将故事写成我们',
 '对的时间点',
 'As I Believe',
 '我们很好',
 '进阶',
 '飞云之下',
 'Despacito 缓缓',
 '圣所',
 '伟大的渺小',
 '穿越',
 '四点四十四',
 '我继续',
 '剪云者',
 '黑夜问白天',
 '丹宁执着',
 '身为风帆',
 '小瓶子',
 'Until the Day',
 '黑夜问白天',
 '小瓶子',
 '伟大的渺小

In [5]:
song_infos = []

url_baike = "https://baike.baidu.com"

# 四种情况
# case 1:正确找到对应歌曲百科
# case 2:找到多义词的百科界面-->需跳转到多义词的列表界面-->找到对应歌曲百科
# case 3:找到多义词的列表界面-->找到对应歌曲百科
# case 4:找不到百科

# 找到资料的前一个a标签
def is_info_pre_a(tag):
    return tag.has_attr('name') and tag['name'] == 'lemma-summary'

pattern = r'^共\d+个义项'
# 若有多义词，则必存在指向多义词的a标签
def is_multi_a(tag):
    return tag.has_attr('href') and re.search(pattern=pattern, string=tag.text)

# 从多义词列表中找到歌曲链接
def is_target_a(tag):
    return tag.has_attr('href') and "林俊杰" in tag.text

# case 1
def get_baike(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "lxml")
    try:
        d = soup.find(is_info_pre_a).next_sibling
        if "林俊杰" in d.get_text():
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")
        else:
            # case 2
            d = soup.find(is_multi_a)
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")
            # 进入case 3
            d = soup.find(is_target_a)
            if d is None:
                # 说明找不到该歌曲对应百科
                print("找不到对应百科")
            else:
                res = requests.get(url=url_baike+d['href'])
                soup = BeautifulSoup(res.text, "lxml")

                d = soup.find(is_info_pre_a).next_sibling
                song_infos.append([name, d.get_text()])
                print("添加歌曲信息成功")
    except:
        # case 3 or case 4
        # 如果能找到有林俊杰的a标签则是case 3
        d = soup.find(is_target_a)
        if d is None:
            # 说明找不到该歌曲对应百科
            print("找不到对应百科")
        else:
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")

            d = soup.find(is_info_pre_a).next_sibling
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")

num = 1
for name in song_names:
    print("["+str(num) + "/" + str(len(song_names))+"] "+name+":")
    get_baike(url=url_baike+"/item/"+name)

    num += 1
    n = random.randint(0, 10)
    time.sleep(n * 0.1)
    



[1/304] The Show:
添加歌曲信息成功
[2/304] 愿与愁:
添加歌曲信息成功
[3/304] 逆光白:
添加歌曲信息成功
[4/304] 孤独娱乐:
添加歌曲信息成功
[5/304] 梦不凌乱:
添加歌曲信息成功
[6/304] 自画像:
添加歌曲信息成功
[7/304] 谢幕:
添加歌曲信息成功
[8/304] 如果我还剩一件事情可以做:
添加歌曲信息成功
[9/304] 黑色泡沫:
添加歌曲信息成功
[10/304] 你都在:
添加歌曲信息成功
[11/304] 一时的选择:
添加歌曲信息成功
[12/304] Castle in the air:
添加歌曲信息成功
[13/304] 7千3百多天:
添加歌曲信息成功
[14/304] 愿与愁:
添加歌曲信息成功
[15/304] In The Joy:
添加歌曲信息成功
[16/304] 谢幕:
添加歌曲信息成功
[17/304] Intro:
添加歌曲信息成功
[18/304] Too Bad:
添加歌曲信息成功
[19/304] 身后:
添加歌曲信息成功
[20/304] 想见你想见你想见你:
添加歌曲信息成功
[21/304] 恨幸福来过:
添加歌曲信息成功
[22/304] 慢慢喜欢你:
添加歌曲信息成功
[23/304] You Are The Reason:
添加歌曲信息成功
[24/304] 咖啡+交换余生:
添加歌曲信息成功
[25/304] 圣所:
添加歌曲信息成功
[26/304] 生生:
添加歌曲信息成功
[27/304] 我还年轻 我还年轻:
添加歌曲信息成功
[28/304] 你啊你啊:
添加歌曲信息成功
[29/304] 那女孩对我说:
添加歌曲信息成功
[30/304] 落在生命里的光:
添加歌曲信息成功
[31/304] 是你:
添加歌曲信息成功
[32/304] 达尔文:
添加歌曲信息成功
[33/304] 7千3百多天:
添加歌曲信息成功
[34/304] 无拘:
添加歌曲信息成功
[35/304] From The Ashes:
添加歌曲信息成功
[36/304] At Least I Had You:
添加歌曲信息成功
[37/304] You're My Superhero:
找不到对应百科
[38/304] Positive Procrastina

In [6]:
song_infos

[['The Show',
  '《The Show》是由Steve Aoki、Carl Ryden、James Daniel Lewis作词，Steve Aoki、Carl Ryden作曲，Steve Aoki、林俊杰演唱的一首歌曲，发行于2023年5月26日。 [1]'],
 ['愿与愁',
  '《愿与愁》是林俊杰演唱的歌曲，由小寒作词，林俊杰作曲，于2023年4月14日以单曲形式发行 [2]，收录在林俊杰同年4月21日发行的专辑《重拾_快乐》中 [3]。'],
 ['逆光白', '《逆光白》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于林俊杰于2023年4月21日发行的专辑《重拾_快乐》中 [3]。'],
 ['孤独娱乐',
  '《孤独娱乐》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑《重拾_快乐》中，作为专辑第二主打歌曲。 [1]'],
 ['梦不凌乱', '《梦不凌乱》是由方文山作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑《重拾_快乐》中。 [1]'],
 ['自画像', '《自画像》是由林怡凤作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑《重拾_快乐》中。 [1]'],
 ['谢幕',
  '《谢幕》是林俊杰演唱的歌曲，致敬已故好友林秋离，由林怡凤、林秋离作词，林俊杰作曲，发行于2023年3月25日 [1]，收录在林俊杰的专辑《重拾_快乐》中。 [2]'],
 ['如果我还剩一件事情可以做',
  '《如果我还剩一件事情可以做 》是由小寒作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑《重拾_快乐》中。 [1]'],
 ['黑色泡沫', '《黑色泡沫》是由周信廷、杨彤作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的《重拾_快乐》专辑中。 [1]'],
 ['你都在',
  '《你都在》是由林怡凤、Rude Cat作词，林俊杰、Rude Cat作曲，林俊杰演唱的一首歌曲，《In The Joy》中文版，收录于2023年4月21日发行的专辑《重拾_快乐》中。 [1]'],
 ['一时的选择', '《一时的选择》是由蔡宥绮作词、作曲，林俊杰演唱的一首歌曲，收录于2023年4月21日发行的《重拾_快乐》专辑中。 [1]'],
 [

In [7]:
# 将歌曲信息添加进文件中保存
df = pd.DataFrame(columns=["歌曲名称","歌曲信息"], data=song_infos)
df.to_csv("song_infos.csv")

In [8]:
df

,歌曲名称,歌曲信息
0,The Show,《The Show》是由Steve Aoki、Carl Ryden、James Daniel...
1,愿与愁,《愿与愁》是林俊杰演唱的歌曲，由小寒作词，林俊杰作曲，于2023年4月14日以单曲形式发行 ...
2,逆光白,《逆光白》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于林俊杰于2023年4月21日发行...
3,孤独娱乐,《孤独娱乐》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑...
4,梦不凌乱,《梦不凌乱》是由方文山作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑...
...,...,...
287,女儿家,歌曲《女儿家》，由林俊杰演唱，张思尔填词，林俊杰谱曲，收录在专辑《乐行者》中。
288,星空下的吻,《星空下的吻》是新加坡歌手林俊杰演唱的歌曲，由张思尔、林秋离作词，林俊杰作曲，收录在林俊杰2...
289,让我心动的人,歌曲《让我心动的人》，由林俊杰演唱，张思尔填词，林俊杰谱曲，收录在专辑《乐行者》中。
290,会有那么一天,《会有那么一天》是新加坡歌手林俊杰演唱的歌曲，由张思尔作词，林俊杰作曲，收录在林俊杰2003...
